# Welcome to our Linear Regression Workshop!

In this notebook, we will first discuss what regression is, and how it mathematically looks like. Then, we will dive deep into the implementation of linear regression using Python and some related libraries. After having a total understanding of the concept, we will solve a real-world problem using the model we generated and evaluate its performance. 

Throughout the workshop, we will be learning a lot of new concepts, their importance, and how they are used in real-life scenarios. Let's get started!

First, let us understand what "regression" and "linear regression" mean.

## Regression
- Regression is a statistical technique that relates a dependent variable (denoted as Y) to one or more independent variables (denoted as X).

- Regression models are able to show whether the changes observed in the dependent variable are associated with changes in the independent variables.

- There are many different regression models, such as "linear regression", "logistic regression", "polynomial regression", "ridge regression", and "lasso regression", and many others.

- In this notebook, we will focus on linear regression, which models the relationship between one dependent variable and one or more independent variables using a straight line, hence the name "linear" regression. 

### Linear Regression
- Linear regression functions are of the form `Y = wX + b`, where

    `Y`: dependent variable vector, what we are trying to guess

    `w`: weight vector, the importance of each independent variable. We will refer to weights as `coefficients` in this workshop, to make the code more understandable.

    `X`: independent variable vector, what we base our predictions on

    `b`: bias, helps with fitting the model better. We will refer to bias as `intercept` in this workshop, again to make the code more understandable.

- The purpose of `finding` the most accurate model requires us to find the best parameters `w` and `b` such that our regression line is a perfect fit for our dataset. That is, we try many many many different `w` and `b` values to find the line that helps us the most. But how do we find them?  

### Mean Squared Error (MSE)

- MSE is a very common measure of the quality of an estimator. It's used to evaluate the accuracy of a linear regression model by calculating the average of the squares of the errors (difference between actual and predicted results).

- Formula:

$$
\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (Y_i - \hat{Y}_i)^2
$$

- The end goal is to minimize MSE to improve the accuracy of the model.

### Gradient Descent

- Gradient Descent is an optimization algorithm used to minimize the cost function (MSE in our case), by iteratively adjusting model parameters. The update rule for each parameter $\theta$ is:

$$
\theta := \theta - \alpha \frac{\partial J(\theta)}{\partial \theta}
$$

where

$$
$$

- $\theta$ : the parameter we are trying to optimize (w or b for linear regression)

- $\alpha$ : learning rate, a hyperparameter that determines the step size at each iteration while moving toward a minimum cost function. Small alpha ensures more precise updates, but training will take more time with smaller alpha (since changes will be very minimal, but very accurate).

- $J(\theta)$ : the cost function, MSE in our case.

- $\frac{\partial J(\theta)}{\partial \theta}$ : Gradient of the cost function with respect to the parameter. It measures how much J changes with a small change in $\theta$. The gradient points in the direction of the steepest increase of the cost function.

### Now that we know what linear regression looks like, let us start implementing it using Python!

Let's import the necessary libraries first.

In [ ]:
import numpy as np
import sklearn.metrics
import joblib

Now, let's implement the Linear Regression class using Python together!

In [ ]:
class LinearRegression:

    def __init__(
        self,
        method="normal_equation",
        learning_rate=0.01,
        epochs=1000,
        regularization=None,
        alpha=0.01,
    ):
        self.coefficients = None
        self.intercept = None
        self.method = method
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.regularization = regularization
        self.alpha = alpha  # Regularization strength

    def fit(self, X, y):
        """
            Fit the model based on the method of choice.
        """
        X = np.array(X)
        y = np.array(y)

        if self.method == "normal_equation":
            self._fit_normal_equation(X, y)

        elif self.method == "gradient_descent":
            self._fit_gradient_descent(X, y)

        else:
            raise ValueError("Method must be 'normal_equation' or 'gradient_descent'")

    def _fit_normal_equation(self, X, y):
        """
            Fit model directly according to formula Y = w*X + b where w is weight (coefficients) and b is bias (intercept).
            This method does not use iteration like gradient descent, but uses the normal equation.
        """
        X_b = np.c_[(np.ones(X.shape[0], 1)), X]  # X_b is the bias term
        if self.regularization == "ridge":
            L = self.alpha * np.eye(X_b.shape[1])
            L[0, 0] = 0  # do not regularize the intercept term
            theta_best = np.linalg.inv(X_b.T.dot(X_b) + L).dot(X_b.T).dot(y)
        else:
            theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
        self.intercept = theta_best[0]
        self.coefficients = theta_best[1:]

    def _fit_gradient_descent(self, X, y):
        """
            Fit model using gradient descent
        """
        X_b = np.c_[np.ones((X.shape[0], 1)), X]  # X_b is the bias term
        m = len(y)
        theta = np.random.randn(X_b.shape[1])  # random initialization of theta

        for epoch in range(self.epochs):
            gradients = 2 / m * X_b.T.dot(X_b.dot(theta) - y)
            if self.regularization == "lasso":
                gradients += self.alpha * np.sign(theta)
            elif self.regularization == "ridge":
                gradients += self.alpha * theta
                gradients[0] -= self.alpha * theta[0]   # do not regularize the intercept term
            theta -= self.learning_rate * gradients

        self.intercept = theta[0]
        self.coefficients = theta[1:]

    def predict(self, X):
        """
            Calculate Y given the X value using the model.
        """
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        return X_b.dot(np.r_[self.intercept, self.coefficients])

    def mean_squared_error(self, y_true, y_pred):
        """
            Calculate mean squared error according to the formula.
        """
        return np.mean((y_true - y_pred) ** 2)

    def r2_score(self, y_true, y_pred):
        """
            Determine the proportion of variance in Y that can be explained by X.
            In other words, a measure of how well the data fits our model.
        """
        total_variance = np.var(y_true) * len(y_true)
        explained_variance = total_variance - np.sum((y_true - y_pred) ** 2)
        return explained_variance / total_variance

    """
        Following two functions are for saving the model for later use.
        If you want to test your model on other datasets, you can save the model to your local. 
    """
    def save_model(self, filepath):
        joblib.dump(self, filepath)

    def load_model(self, filepath):
        return joblib.load(filepath)


This class definition for Linear Regression covers everything we have discused in theory, and further implements other metrics for accuracy calculations and optimizations.

### Now that we have a fully functioning model in our hands, let's put it into use!

We are going to use a very famous dataset called `California Housing`, which is a very famous dataset by Carneige Mellon University. Let's see its desription by loading it first. 

In [ ]:
from sklearn.datasets import fetch_california_housing

california_housing = fetch_california_housing(as_frame=True)

In [ ]:
print(california_housing.DESCR) # see the official description of our dataset